# Data Collection

This analysis will include data from three sources:
1. **Outage Data**: Major US electrical outages from 2018 - 2023 as recorded by the Department of Energy (DOE)(https://www.oe.netl.doe.gov/OE417_annual_summary.aspx)
2. **Weather Data**: Local weather at the time of the outage event as recorded by the National Oceanic and Atomospheric Administration (NOAA) (https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted)
3. **Energy Demand Data**: Levels of energy demand at the time and location of the outage event as recorded by the Energy Information Administration (EIA) (https://www.eia.gov/electricity/gridmonitor/dashboard/electric_overview/regional/REG-CAL)

## Imports

In [1]:
# Import necessary modules to collect data

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os
import json
import requests
import statistics
import time


## Outage Data

In [3]:
# The outage data is formatted as a separate excel spreadsheet for each year.  
# The code below imports each spreadsheet from 2018 - 2023 and combines all the separate excel sheets into one dataframe.
# Note: xlrd needs to be installed in order to execute the code below

# Create an empty list to store the dataframes for each year
outage_dfs = []

# Create a dataframe for each year by looping through years 2018 - 2023
for year in range(2018, 2024):
    start_row = 2
    df = pd.read_excel(f'../Data-Wrangling/Raw-Data/Outage-Data-Raw/{year}_Annual_Summary.xls', skiprows = start_row - 1)
    outage_dfs.append(df)

# Concatenate all the dataframes stored in the dfs list to form a final combined outage dataframe
outage_df = pd.concat(outage_dfs, ignore_index = True)

# Some excel sheets had the column name "Month" while others had the column name "Event Month".
# The code below combines this into a single column named "Event Month"
outage_df['Month'].fillna(outage_df['Event Month'], inplace = True)
outage_df.drop(columns = ['Event Month'], inplace = True)
outage_df.rename(columns = {'Month' : 'Event Month'}, inplace = True)

# The dataset includes 'unknown' values that are not registering as null values.
# The code below converts 'unknown' values to null values
outage_df.replace('.*Unknown.*', pd.NA, inplace = True, regex = True)

# The code below converts columns to the proper data type

    # 'Date Event Began' and 'Date of Restoration' should be converted to datetimes:
outage_df['Date Event Began'] = pd.to_datetime(outage_df['Date Event Began'])
outage_df['Date of Restoration'] = pd.to_datetime(outage_df['Date of Restoration'], format = '%m/%d/%Y', errors = 'coerce')

    # 'Time Event Began' and 'Time of Restoration' can be added into their respective 'Date' columns and converted to datetimes
    # Convert 'Time Event Began' to a timedelta, combine with 'Date Event Began', and rename the column 'Datetime Event Began'
outage_df['Timedelta_begin'] = outage_df['Time Event Began'].apply(lambda x: pd.to_timedelta(x.strftime('%H:%M:%S')))
outage_df['Datetime Event Began'] = outage_df['Date Event Began'] + outage_df['Timedelta_begin']

    # Convert 'Time of Restoration' to a timedelta, combine with 'Date of Restoration', and rename the column 'Datetime of Restoration'
    # First, the missing time values must be replaced with a default time
default_time = '00:00:00'
outage_df['Time of Restoration'].fillna(default_time, inplace=True)

    # Two of the 'Time of Restoration' values are improperly formatted because of incomplete information.  The code below corrects this
outage_df.iloc[1956, 4] = '16:41:00'
outage_df.iloc[1956, 4]

outage_df.iloc[1983, 4] = '00:00:00'
outage_df.iloc[1983, 4]

    # The code below completes the conversion of the 'Time of Restoration' column and creates a new column for 'Datetime of Restoration'
outage_df['Time of Restoration'] = pd.to_datetime(outage_df['Time of Restoration'], format='%H:%M:%S').dt.time
outage_df['Timedelta_restoration'] = outage_df['Time of Restoration'].apply(lambda x: pd.Timedelta(hours=x.hour, minutes=x.minute, seconds=x.second))
outage_df['Datetime of Restoration'] = outage_df['Date of Restoration'] + outage_df['Timedelta_restoration']

    # The event month, original date / time columns, and intermediate timedelta columns can be dropped and the columns can be rearranged to have the datetimes as the first columns
outage_df.drop(columns = ['Event Month','Date Event Began','Time Event Began', 'Date of Restoration', 'Time of Restoration', 'Timedelta_begin','Timedelta_restoration'], inplace = True)
outage_df = outage_df[['Datetime Event Began','Datetime of Restoration','Area Affected','NERC Region','Alert Criteria','Event Type','Demand Loss (MW)','Number of Customers Affected']]
outage_df.dtypes

    # 'Demand Loss' and 'Number of Customers Affected' should be converted to numeric values
outage_df['Demand Loss (MW)'] = pd.to_numeric(outage_df['Demand Loss (MW)'], errors = 'coerce')
outage_df['Number of Customers Affected'] = pd.to_numeric(outage_df['Number of Customers Affected'], errors = 'coerce')

# Some of the rows represent events that are not true outages, these rows should be deleted from  the dataset
    # Some rows represent events that are warnings to the public to reduce energy consumption; these rows will be dropped since they are not true outages
rows_to_drop = ~outage_df['Alert Criteria'].str.contains('Public appeal to reduce the use of electricity')
outage_df = outage_df[rows_to_drop]

    # Rows that have no demand loss (either Null or 0) are not outages and will be dropped
outage_df = outage_df[(outage_df['Demand Loss (MW)'] != 0) & (~outage_df['Demand Loss (MW)'].isna())].reset_index(drop = True)

# 'Area Affected' Column includes information related to the state, county, and in some entries the power company responsible for that area
# The code below will create a new column for 'State Affected' using the information from the 'Area Affected' Column
list_of_states = ['Alabama', 'Alaska', 'American Samoa', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Guam', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Minor Outlying Islands', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Northern Mariana Islands', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'U.S. Virgin Islands', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
conditions = []
for state in list_of_states:
    conditions +=[(outage_df['Area Affected'].str.contains(state))]
outage_df['State Affected'] = np.select(conditions, list_of_states, default = 'Other')
# The code below assigns states categorized as 'Other' to their appropriate state
outage_df[outage_df['State Affected'] == 'Other'] # To see rows where 'State Affected' is 'Other'
outage_df.iloc[108, 8] = 'New York' # Area affected is Western NY, assigning State to New York
outage_df.iloc[417,8] = 'North Dakota' # Area affected is Upper Greater Plains Region, assigning state to North Dakota
outage_df.loc[outage_df['Area Affected'] == 'LUMA Energy', 'State Affected'] = 'Puerto Rico' # LUMA Energy serves Puerto Rico
outage_df.iloc[426, 8] = 'North Dakota' # Otter Tail Power Co serves North Dakota
outage_df = outage_df[['Datetime Event Began','Datetime of Restoration','Area Affected','State Affected','NERC Region','Alert Criteria','Event Type','Demand Loss (MW)','Number of Customers Affected']]
outage_df

WARNING *** file size (96452) not 512 + multiple of sector size (512)
WARNING *** file size (120004) not 512 + multiple of sector size (512)
WARNING *** file size (133828) not 512 + multiple of sector size (512)
WARNING *** file size (134340) not 512 + multiple of sector size (512)


,Datetime Event Began,Datetime of Restoration,Area Affected,State Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected
0,2018-01-02 10:00:00,2018-02-12 08:00:00,New York: Niagara County;,New York,NPCC,Fuel supply emergencies that could impact elec...,Fuel Supply Deficiency,675.0,NaN
1,2018-01-02 06:45:00,2018-01-02 09:00:00,North Carolina: South Carolina:,North Carolina,SERC,System-wide voltage reductions of 3 percent or...,Severe Weather,14998.0,NaN
2,2018-01-12 13:08:00,2018-01-12 14:53:00,"Michigan: Midland County, Genesee County;",Michigan,RF,Cyber event that causes interruptions of elect...,System Operations,41.0,23007.0
3,2018-02-04 13:42:00,2018-02-04 15:25:00,California:,California,WECC,Physical attack that could potentially impact ...,Vandalism,9760.0,0.0
4,2018-02-08 13:25:00,2018-02-08 13:31:00,California:,California,WECC,Electrical System Separation (Islanding) where...,System Operations,30.0,10900.0
...,...,...,...,...,...,...,...,...,...
422,2023-12-18 13:11:00,2023-12-18 13:23:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,115.0,170316.0
423,2023-12-19 00:41:00,2023-12-19 00:47:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,77.0,50204.0
424,2023-12-23 00:43:00,2023-12-23 00:48:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,65.0,37850.0
425,2023-12-25 19:56:00,2023-12-25 20:01:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,105.0,36100.0


## Weather Data

In [5]:
# I will use the National Centers for Environmental Information API to pull weather data at the time and location of each outage event.
# The API overview is located here: https://www.ncdc.noaa.gov/cdo-web/webservices/v2#gettingStarted

# The code below pulls the NOAA API token from a local .env file.  API token can be requested here: https://www.ncdc.noaa.gov/cdo-web/token
from dotenv import load_dotenv
load_dotenv()
token = os.getenv('NOAA_token')
# print(token) # This checks that the token was pulled correctly

# Setting a variable "header" equal to the token to be included in the GET request
headers = {"token": token}

# Pulling data from the API regarding state location ids. Creating a dictionary of state location ids to use in subsequent GET requests.
url_1 = "https://www.ncei.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ST&limit=100"
r_1 = requests.get(url_1, headers = headers)
json_data_1 = r_1.json()
dict_data_1 = dict(json_data_1)
# print(dict_data_1)
FIPSID_dict = {dict['name']:dict['id'] for dict in dict_data_1['results']}
# # print(state_FIPSID)

# # Pulling data from the API regarding US territory location ids. Adding these ids to the US_TERR_FIPSID dictionary.
url_2 = "https://www.ncei.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=US_TERR&limit=100"
r_2 = requests.get(url_2, headers = headers)
json_data_2 = r_2.json()
dict_data_2 = dict(json_data_2)
# print(dict_data_2)
US_TERR_FIPSID = {dict['name']:dict['id'] for dict in dict_data_2['results']}
# # print(US_TERR_FIPSID)

# Combining the FIPSID dictionaries
FIPSID_dict.update(US_TERR_FIPSID)
# print(FIPSID_dict)

In [110]:
# Setting request parameters to pull relevant weather data from the API.
dataset_id = "GHCND" # This id corresponds to the data set with historical daily climate observations. Documentation for this dataset is here: https://www.ncei.noaa.gov/pub/data/ghcn/daily/readme.txt

# Below are the datatype ids of the specific observations I would like to pull from each event
avg_temp_datatype_id = "TAVG" # Returns avg temperature for the date and location specified in tenths of degrees Celsius
avg_wind_speed_datatype_id = "AWND" # Returns avg windspeed for the date/location specified in tenths of meters/second
high_wind_speed_datatype_id = "WSFI" # Returns highest instanteneous windspeed for date/location specified in tenths of meters/second
precip_datatype_id = "PRCP" # Returns precipitation for date/location in tenths of mm

# Creating a function for a GET request for weather data for each event in the outage_df. 
# This code pulls the four datatypes listed above from all weather stations within the affected state, averages them over all the stations, and returns a single value for each datatype which will be incorporated in the outage_df
def weather_data_request(event_index):
    
    '''
    This function will issue a request to the NOAA CDO API for the four parameters described for a specific event in the outage_df.
    The only argument/parameter needed is the associated index number from the outage_df dataset above.
    '''
    
    start_date = outage_df['Datetime Event Began'][event_index].date()   # This will pull the start date from the outage dataset
    end_date = start_date                                                # The end date should correspond to the start date because we are only interested in the weather at the time the outage occurred
    location_id = FIPSID_dict[outage_df['State Affected'][event_index]]  # This pulls the affected state from the outage_df and the associated FIPSID from the FIPSID_dict
    url_3 = f"https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid={dataset_id}&locationid={location_id}&startdate={start_date}&enddate={end_date}&datatypeid={avg_temp_datatype_id}&datatypeid={avg_wind_speed_datatype_id}&datatypeid={high_wind_speed_datatype_id}&datatypeid={precip_datatype_id}&limit=1000"

    max_retries = 3   # This api will often return a 503 error.  Creating a while loop to retry the request 3 times if a 503 error is returned
    retries = 0
    while retries <= max_retries:
        try:
            r_3 = requests.get(url_3, headers = headers)
            # print(r_3)
            if r_3.text.strip():
                json_data_3 = r_3.json()
                dict_data_3 = dict(json_data_3)
                # print(dict_data_3)
                if dict_data_3 == {}:
                    return 'No Data'
                else:
                    # Checks if this data was available and then calculates avg temp over all stations in the state during date of event
                    if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='TAVG']!= []:
                        avg_temp_in_C = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='TAVG'])
                        avg_temp_in_F = ((avg_temp_in_C/10)*(9/5))+32 # Converts from tenth degree Celsius to Farenheit
                    else:
                        avg_temp_in_F = 'No Data'
        
                    # Checks if this data was available and then calculates avg wind speed over all stations in state during date of event
                    if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='AWND']!= []:
                        avg_windspeed_meterspersecond = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='AWND'])/10 # divide by 10 to convert to meters per second
                        avg_windspeed_mph = avg_windspeed_meterspersecond * 2.2369 # converts meters per second to miles per hour
                    else:
                        avg_windspeed_mph = 'No Data'
        
                    # Checks if this data was available and then calculates average highest wind speed in state during date of event
                    if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='WSFI']!= []:
                        high_windspeed_meterspersecond = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='WSFI'])/10 # divide by 10 to convert to meters per second
                        high_windspeed_mph = high_windspeed_meterspersecond * 2.2369 # converts meters per second to miles per hour
                    else:
                        high_windspeed_mph = 'No Data'
        
                    # Checks if this data was available and then calculates average precipitation in state during date of event
                    if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='PRCP']!= []:
                        avg_precip_mm = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='PRCP'])/10 # divide by 10 to convert to mm
                    else:
                        avg_precip_mm = 'No Data'
                    
                    return [avg_temp_in_F, avg_windspeed_mph, high_windspeed_mph, avg_precip_mm]
            else:
                return 'No Data'
        
        except requests.exceptions.HTTPError as e:
            if response.status_code == 503:
                time.sleep(30)
                retries += 1
            else:
                time.sleep(10)
                retries += 1
               

In [96]:
# This cell was  used to manually add index 407 - 411 to the weather data dictionary

# start_date = outage_df['Datetime of Restoration'][411].date()   # This will pull the start date from the outage dataset
# end_date = start_date                                                # The end date should correspond to the start date because we are only interested in the weather at the time the outage occurred
# location_id = FIPSID_dict[outage_df['State Affected'][411]]  # This pulls the affected state from the outage_df and the associated FIPSID from the FIPSID_dict
# url_3 = f"https://www.ncei.noaa.gov/cdo-web/api/v2/data?datasetid={dataset_id}&locationid={location_id}&startdate={start_date}&enddate={end_date}&datatypeid={avg_temp_datatype_id}&datatypeid={avg_wind_speed_datatype_id}&datatypeid={high_wind_speed_datatype_id}&datatypeid={precip_datatype_id}&limit=1000"

# max_retries = 3
# retries = 0
# while retries <= max_retries:
#     try:
#         r_3 = requests.get(url_3, headers = headers)
#         if r_3.text.strip():
#             json_data_3 = r_3.json()
#             dict_data_3 = dict(json_data_3)
#             # print(dict_data_3)
#             if dict_data_3 == {}:
#                 print( 'No Data')
#             else:
#                 # Checks if this data was available and then calculates avg temp over all stations in the state during date of event
#                 if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='TAVG']!= []:
#                     avg_temp_in_C = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='TAVG'])
#                     avg_temp_in_F = ((avg_temp_in_C/10)*(9/5))+32 # Converts from tenth degree Celsius to Farenheit
#                 else:
#                     avg_temp_in_F = 'No Data'
    
#                 # Checks if this data was available and then calculates avg wind speed over all stations in state during date of event
#                 if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='AWND']!= []:
#                     avg_windspeed_meterspersecond = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='AWND'])/10 # divide by 10 to convert to meters per second
#                     avg_windspeed_mph = avg_windspeed_meterspersecond * 2.2369 # converts meters per second to miles per hour
#                 else:
#                     avg_windspeed_mph = 'No Data'
    
#                 # Checks if this data was available and then calculates average highest wind speed in state during date of event
#                 if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='WSFI']!= []:
#                     high_windspeed_meterspersecond = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='WSFI'])/10 # divide by 10 to convert to meters per second
#                     high_windspeed_mph = high_windspeed_meterspersecond * 2.2369 # converts meters per second to miles per hour
#                 else:
#                     high_windspeed_mph = 'No Data'
    
#                 # Checks if this data was available and then calculates average precipitation in state during date of event
#                 if [dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='PRCP']!= []:
#                     avg_precip_mm = statistics.mean([dict['value'] for dict in dict_data_3['results'] if dict['datatype']=='PRCP'])/10 # divide by 10 to convert to mm
#                 else:
#                     avg_precip_mm = 'No Data'
                
#                 print( [avg_temp_in_F, avg_windspeed_mph, high_windspeed_mph, avg_precip_mm])
#                 break
#         else:
#             print('No Data')
    
#     except requests.exceptions.HTTPError as e:
#         if response.status_code == 503:
#             retries += 1
#         else:
#             retries += 1

# weather_data_dict[411] = [avg_temp_in_F, avg_windspeed_mph, high_windspeed_mph, avg_precip_mm]

[69.38705882352942, 7.242927931034482, 'No Data', 17.18986784140969]


In [98]:
# Creating a for loop to iterate through the outage_df and pull the associated weather observations for each event. This loop will also append the weather features to the outage_df dataset

# This API is very finniky and will often return a 503 error.  I created a variable to keep track of the last index updated so the for loop can continue from where it left off.
# weather_data_dict = {}
# last_index_updated = 426

for index, row in outage_df.iterrows():
    if last_index_updated is not None and index <= last_index_updated:
        continue
    else:
        weather_data_dict[index] = weather_data_request(index)
        last_index_updated = index

# print(len(weather_data_dict))
# last_index_updated

# Note: needed to manually add data for index 407 through 411 as there is no start date, only a restoration date
    

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
427


426

In [99]:
# last_index_updated
weather_data_dict

{0: [11.261428571428574, 11.499064062499999, 'No Data', 0.8113861386138614],
 1: [18.694594594594594, 4.726666956521738, 'No Data', 0.0004243281471004243],
 2: [18.94470588235294, 15.835709310344825, 'No Data', 8.174585635359117],
 3: [56.09491289198606, 1.11845, 5.59225, 0.005625879043600563],
 4: [56.66064516129032, 'No Data', 'No Data', 0.0011095700416088765],
 5: [48.581575342465754, 4.02642, 20.35579, 0.012889518413597733],
 6: [48.581575342465754, 4.02642, 20.35579, 0.012889518413597733],
 7: [39.871497005988026, 'No Data', 'No Data', 14.564825930372148],
 8: [36.769999999999996, 10.676113636363635, 'No Data', 10.288479262672812],
 9: [65.58500000000001, 5.965066666666667, 'No Data', 6.305194805194805],
 10: [35.42, 10.005043636363636, 'No Data', 0.36826086956521736],
 11: [36.25454545454546, 8.317200909090907, 'No Data', 2.618664047151277],
 12: [37.218947368421055,
  6.407120714285714,
  17.969763333333333,
  0.5495268138801261],
 13: [37.218947368421055,
  6.407120714285714,
 

In [108]:
# Create a dataframe from the weather dictionary and add it to the outage_df
weather_df = pd.DataFrame.from_dict(weather_data_dict,orient = 'index',columns = ['State Avg Temp (F)', 'State Avg Windspeed (mph)', 'State High Windspeed (mph)', 'State Avg Precipitation (mm)'])
outage_df = pd.concat([outage_df, weather_df], axis = 1)
outage_df

,Datetime Event Began,Datetime of Restoration,Area Affected,State Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,State Avg Temp (F),State Avg Windspeed (mph),State High Windspeed (mph),State Avg Precipitation (mm)
0,2018-01-02 10:00:00,2018-02-12 08:00:00,New York: Niagara County;,New York,NPCC,Fuel supply emergencies that could impact elec...,Fuel Supply Deficiency,675.0,NaN,11.261429,11.499064,No Data,0.811386
1,2018-01-02 06:45:00,2018-01-02 09:00:00,North Carolina: South Carolina:,North Carolina,SERC,System-wide voltage reductions of 3 percent or...,Severe Weather,14998.0,NaN,18.694595,4.726667,No Data,0.000424
2,2018-01-12 13:08:00,2018-01-12 14:53:00,"Michigan: Midland County, Genesee County;",Michigan,RF,Cyber event that causes interruptions of elect...,System Operations,41.0,23007.0,18.944706,15.835709,No Data,8.174586
3,2018-02-04 13:42:00,2018-02-04 15:25:00,California:,California,WECC,Physical attack that could potentially impact ...,Vandalism,9760.0,0.0,56.094913,1.11845,5.59225,0.005626
4,2018-02-08 13:25:00,2018-02-08 13:31:00,California:,California,WECC,Electrical System Separation (Islanding) where...,System Operations,30.0,10900.0,56.660645,No Data,No Data,0.001110
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2023-12-18 13:11:00,2023-12-18 13:23:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,115.0,170316.0,No Data,8.50022,No Data,2.275862
423,2023-12-19 00:41:00,2023-12-19 00:47:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,77.0,50204.0,No Data,5.59225,No Data,0.332813
424,2023-12-23 00:43:00,2023-12-23 00:48:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,65.0,37850.0,No Data,6.151475,No Data,10.081356
425,2023-12-25 19:56:00,2023-12-25 20:01:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,105.0,36100.0,No Data,5.36856,No Data,0.638182


In [109]:
# Replace 'No Data' with Null values
outage_df.replace('No Data', pd.NA, inplace = True)
outage_df.isna().sum()

Datetime Event Began              5
Datetime of Restoration          55
Area Affected                     0
State Affected                    0
NERC Region                       8
Alert Criteria                    0
Event Type                        4
Demand Loss (MW)                  0
Number of Customers Affected     79
State Avg Temp (F)               58
State Avg Windspeed (mph)        57
State High Windspeed (mph)      339
State Avg Precipitation (mm)      0
dtype: int64

There are several missing values for Avg Temp, Avg Windspeed, and High Windspeed:
- For avg temp: I may fill in with the high temperature for the day or drop them. Depending on if how EDA turns out and if the high temp is available from NOAA
- For Avg Windspeed: I may continue with having null values in the dataset and see how EDA turns out
- For High Windspeed: There are too many missing values.  I will drop the column.

Potential for further data collection - I would like to add columns for the normal values of temperature, precipitation, and windspeed to compare to the actual observations on that day.  Further exploration of the NOAA API is needed to see if this data is available.

In [116]:
# Drop the high windspeed column since there are too many missing values
# outage_df.drop(columns=['State High Windspeed (mph)'], inplace = True)
outage_df

,Datetime Event Began,Datetime of Restoration,Area Affected,State Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,State Avg Temp (F),State Avg Windspeed (mph),State Avg Precipitation (mm)
0,2018-01-02 10:00:00,2018-02-12 08:00:00,New York: Niagara County;,New York,NPCC,Fuel supply emergencies that could impact elec...,Fuel Supply Deficiency,675.0,NaN,11.261429,11.499064,0.811386
1,2018-01-02 06:45:00,2018-01-02 09:00:00,North Carolina: South Carolina:,North Carolina,SERC,System-wide voltage reductions of 3 percent or...,Severe Weather,14998.0,NaN,18.694595,4.726667,0.000424
2,2018-01-12 13:08:00,2018-01-12 14:53:00,"Michigan: Midland County, Genesee County;",Michigan,RF,Cyber event that causes interruptions of elect...,System Operations,41.0,23007.0,18.944706,15.835709,8.174586
3,2018-02-04 13:42:00,2018-02-04 15:25:00,California:,California,WECC,Physical attack that could potentially impact ...,Vandalism,9760.0,0.0,56.094913,1.11845,0.005626
4,2018-02-08 13:25:00,2018-02-08 13:31:00,California:,California,WECC,Electrical System Separation (Islanding) where...,System Operations,30.0,10900.0,56.660645,<NA>,0.001110
...,...,...,...,...,...,...,...,...,...,...,...,...
422,2023-12-18 13:11:00,2023-12-18 13:23:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,115.0,170316.0,<NA>,8.50022,2.275862
423,2023-12-19 00:41:00,2023-12-19 00:47:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,77.0,50204.0,<NA>,5.59225,0.332813
424,2023-12-23 00:43:00,2023-12-23 00:48:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,65.0,37850.0,<NA>,6.151475,10.081356
425,2023-12-25 19:56:00,2023-12-25 20:01:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,105.0,36100.0,<NA>,5.36856,0.638182


## Energy Demand Data

In [161]:
# The code below pulls the EIA API key from a local .env file.  API key can be requested here: https://www.eia.gov/opendata/documentation.php
from dotenv import load_dotenv
load_dotenv()
API_key = os.getenv('EIA_API_key')
# print(API_key) # This checks that the API key was pulled correctly

# Electricity generation is separated into subregions. Below is a dictionary mapping the electricy generating subregions to the states they serve.  
# This dictionary will be used to map the data pulled from the API to the outage dataframe
elec_subba_by_state = {
    'Minnesota': ['0001', '0027', '0035', '8910', 'WAUE'],
    'North Dakota': ['0001', '0027', '8910', 'WAUE'],
    'South Dakota': ['0001', '0027', 'WAUE'],
    'Wisconsin': ['0001', '0004', '0027', '0035', '8910'],
    'Illinois': ['0004', '0006', '0035', '8910', 'CE'],
    'Indiana': ['0004', '0006', '8910', 'AEP'],
    'Michigan': ['0004', '0006', '8910', 'AEP', 'ATSI'],
    'Ohio': ['0006', '8910', 'AEP', 'AP', 'ATSI', 'DAY', 'DEOK'],
    'Iowa': ['0027', '0035', '8910', 'WAUE'],
    'Maine': ['4001'],
    'New Hampshire': ['4002'],
    'Vermont': ['4003'],
    'Connecticut': ['4004'],
    'Rhode Island': ['4005'],
    'Massachusetts': ['4006', '4007', '4008'],
    'Arkansas': ['8910', 'CSWS'],
    'Kentucky': ['8910', 'AEP', 'DEOK', 'EKPC'],
    'Louisiana': ['8910', 'CSWS'],
    'Mississippi': ['8910'],
    'Missouri': ['8910', 'CSWS', 'EDE', 'INDN', 'KACY', 'KCPL', 'MPS', 'SPRM'],
    'New Jersey': ['AE', 'JC', 'PS', 'RECO'],
    'Tennessee': ['AEP'],
    'Virginia': ['AEP', 'DOM', 'DPL'],
    'West Virginia': ['AEP', 'AP'],
    'Maryland': ['AP', 'BC', 'DPL', 'PEP'],
    'Pennsylvania': ['AP', 'DUQ', 'ME', 'PE', 'PL', 'PN'],
    'Texas': ['COAS', 'CSWS', 'EAST', 'FWES', 'NCEN', 'NRTH', 'SCEN', 'SOUT', 'SPS', 'WEST'],
    'Kansas': ['CSWS', 'EDE', 'KACY', 'KCPL', 'SECI', 'WR'],
    'Oklahoma': ['CSWS', 'EDE', 'GRDA', 'OKGE', 'WFEC'],
    'New Mexico': ['CYGA', 'Frep', 'Jica', 'KAFB', 'KCEC', 'LAC', 'NTUA', 'PNM', 'SPS', 'TSGT'],
    'Delaware': ['DPL'],
    'Nebraska': ['LES', 'NPPD', 'OPPD', 'WAUE'],
    'Arizona': ['NTUA', 'TSGT'],
    'Utah': ['NTUA'],
    'California': ['PGAE', 'SCE', 'SDGE'],
    'New York': ['RECO', 'ZONA', 'ZONB', 'ZONC', 'ZOND', 'ZONE', 'ZONF', 'ZONG', 'ZONH', 'ZONI', 'ZONJ', 'ZONK'],
    'Colorado': ['TSGT'],
    'Wyoming': ['TSGT'],
    'Nevada': ['VEA'],
    'Montana': ['WAUE'],
    'Florida': ['FPL', 'GP', 'TECO', 'DEF', 'OUC', 'JEA', 'FPL'],
    'Puerto Rico': ['LUMA'],
    'Alabama': ['AP'],
    'Alaska': ['AELP'],
    'Georgia': ['GP'],
    'Hawaii': ['HE'],
    'Idaho': ['IP'],
    'Louisiana': ['EL'],
    'Mississippi': ['MP'],
    'North Carolina': ['DEC', 'NCEMC'],
    'South Carolina': ['SCEG'],
    'Washington': ['PSE'],
    'Oregon': ['PGE']
}



In [159]:
# Creating a function to pull energy demand data for each row in the outage dataframe

def energy_demand_data_request(event_index):
    start_date = outage_df['Datetime Event Began'][event_index].date()
    end_date = start_date
    
    # The below request will pull the electrical demand data for the day of the outage event
    url_4 = f'https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/?api_key={API_key}&frequency=local-hourly&data[0]=value&start={start_date}T00:00:00-07:00&end={end_date}T00:00:00-07:00&offset=0&length=5000'
    
    try:
        r_4 = requests.get(url_4)
        print(r_4)
        if r_4.text.strip():
            json_data_4 = r_4.json()
            dict_data_4 = dict(json_data_4)
            # print(dict_data_4)
            if dict_data_4 == {}:
                return 'No Data'
        else:
            return 'No Data'
            
    except requests.exceptions.HTTPError as e:
        return 'error:' + e

    daily_demand_mwh = sum([int(dict['value']) if dict['subba'] in elec_subba_by_state[outage_df['State Affected'].loc[event_index]] else 0 for dict in dict_data_4['response']['data']])
    if daily_demand_mwh > 0:
        daily_demand_mwh
    else:
        daily_demand_mwh = 'No Data'
    return daily_demand_mwh

In [171]:
# This code was used to pull index 407-411 manually since there is no beginning date for these events

# manual_entry_index = 411
# start_date = outage_df['Datetime of Restoration'][manual_entry_index].date()
# end_date = start_date

# # The below request will pull the electrical demand data for the day of the outage event
# url_4 = f'https://api.eia.gov/v2/electricity/rto/region-sub-ba-data/data/?api_key={API_key}&frequency=local-hourly&data[0]=value&start={start_date}T00:00:00-07:00&end={end_date}T00:00:00-07:00&offset=0&length=5000'

# try:
#     r_4 = requests.get(url_4)
#     print(r_4)
#     if r_4.text.strip():
#         json_data_4 = r_4.json()
#         dict_data_4 = dict(json_data_4)
#         print(dict_data_4)
#         if dict_data_4 == {}:
#             print('No Data')
#     else:
#         print('No Data')
        
# except requests.exceptions.HTTPError as e:
#     print('error:' + e)

# # The code below will use the data from the request to calculate the day's electrical demand in the affected state on the day of the outage event
# daily_demand_mwh = sum([int(dict['value']) for dict in dict_data_4['response']['data'] if dict['subba'] in elec_subba_by_state[outage_df['State Affected'].loc[manual_entry_index]]])
# if daily_demand_mwh > 0:
#     print(daily_demand_mwh)
# else:
#     print('No Data')

# energy_demand_data_dict[manual_entry_index] = daily_demand_mwh

In [170]:
# This for loop will iterrate through the outage_df and request the energy demand data for the specific date, calculate the daily demand for the affected state and store it in a dictionary

# energy_demand_data_dict = {}
last_index_updated = 411

for index, row in outage_df.iterrows():
    if last_index_updated is not None and index <= last_index_updated:
        continue
    else:
        energy_demand_data_dict[index] = energy_demand_data_request(index)
        last_index_updated = index
        
# print(energy_demand_data_dict)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
{0: 'No Data', 1: 'No Data', 2: 'No Data', 3: 'No Data', 4: 'No Data', 5: 'No Data', 6: 'No Data', 7: 'No Data', 8: 'No Data', 9: 'No Data', 10: 'No Data', 11: 'No Data', 12: 'No Data', 13: 'No Data', 14: 'No Data', 15: 'No Data', 16: 'No Data', 17: 'No Data', 18: 'No Data', 19: 'No Data', 20: 'No Data', 21: 'No Data', 22: 'No Data', 23: 'No Data', 24: 37965, 25: 43467, 26: 43467, 27: 'No Data', 28: 'No Data', 29: 34545, 30: 15504, 31: 6842, 32: 'No Data', 33: 'No Data', 34: 8725, 35: 26570, 36: 8061, 37: 8764, 38: 7161, 39: 'No Data', 40: 'No Data', 41: 25041, 42: 4181, 43: 'No Data', 44: 25862, 45: 8031, 46: 24567, 47: 23042, 48: 23451, 49: 63060, 50: 8343, 51: 306, 52: 'No Data', 53: 8383, 54: 5372, 55: 292, 56: 18848, 57: 20142, 58

In [176]:
# The code below will add the energy demand data to the outage_df
energy_demand_df = pd.DataFrame.from_dict(energy_demand_data_dict,orient = 'index',columns = ['State Daily Energy Demand (MWh)'])
outage_df = pd.concat([outage_df, energy_demand_df], axis = 1)
outage_df

,Datetime Event Began,Datetime of Restoration,Area Affected,State Affected,NERC Region,Alert Criteria,Event Type,Demand Loss (MW),Number of Customers Affected,State Avg Temp (F),State Avg Windspeed (mph),State Avg Precipitation (mm),State Daily Energy Demand (MWh)
0,2018-01-02 10:00:00,2018-02-12 08:00:00,New York: Niagara County;,New York,NPCC,Fuel supply emergencies that could impact elec...,Fuel Supply Deficiency,675.0,NaN,11.261429,11.499064,0.811386,No Data
1,2018-01-02 06:45:00,2018-01-02 09:00:00,North Carolina: South Carolina:,North Carolina,SERC,System-wide voltage reductions of 3 percent or...,Severe Weather,14998.0,NaN,18.694595,4.726667,0.000424,No Data
2,2018-01-12 13:08:00,2018-01-12 14:53:00,"Michigan: Midland County, Genesee County;",Michigan,RF,Cyber event that causes interruptions of elect...,System Operations,41.0,23007.0,18.944706,15.835709,8.174586,No Data
3,2018-02-04 13:42:00,2018-02-04 15:25:00,California:,California,WECC,Physical attack that could potentially impact ...,Vandalism,9760.0,0.0,56.094913,1.11845,0.005626,No Data
4,2018-02-08 13:25:00,2018-02-08 13:31:00,California:,California,WECC,Electrical System Separation (Islanding) where...,System Operations,30.0,10900.0,56.660645,<NA>,0.001110,No Data
...,...,...,...,...,...,...,...,...,...,...,...,...,...
422,2023-12-18 13:11:00,2023-12-18 13:23:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,115.0,170316.0,<NA>,8.50022,2.275862,No Data
423,2023-12-19 00:41:00,2023-12-19 00:47:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,77.0,50204.0,<NA>,5.59225,0.332813,No Data
424,2023-12-23 00:43:00,2023-12-23 00:48:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,65.0,37850.0,<NA>,6.151475,10.081356,No Data
425,2023-12-25 19:56:00,2023-12-25 20:01:00,LUMA Energy,Puerto Rico,NaN,Firm load shedding of 100 Megawatts or more im...,- Generator loss or failure,105.0,36100.0,<NA>,5.36856,0.638182,No Data


In [177]:
# Convert cells with 'No Data' to null values
outage_df.replace('No Data', pd.NA, inplace = True)
outage_df.isna().sum()

Datetime Event Began                 5
Datetime of Restoration             55
Area Affected                        0
State Affected                       0
NERC Region                          8
Alert Criteria                       0
Event Type                           4
Demand Loss (MW)                     0
Number of Customers Affected        79
State Avg Temp (F)                  58
State Avg Windspeed (mph)           57
State Avg Precipitation (mm)         0
State Daily Energy Demand (MWh)    111
dtype: int64

There are quite a few null values for daily energy demand, which may affect the analysis.  This will be explored and corrected in EDA.

In [178]:
# Saving the outage_df as a csv file
outage_df.to_csv('outage_df.csv')